In [1]:
#libraries
import numpy as np
import matplotlib.pyplot as plt 
import re
import math
import os
import random
from numpy import genfromtxt
import csv
import string
import re
from numpy import errstate,isneginf,array
import random

## Naive Bayes Classifier

## For Tweets Multiclass Sentiment Analysis (Positive, Negitive, Neutral)


In [2]:
noOfClass = 3  # three classes
dataset_dir = r'Tweets.csv' #directory
stopWords = ['the', 'at', 'of', 'this', 'hence', 'an', 'do', 'us', 'r', 'u', 'ill', 'aa', 'does', 'did', 'im', 'also', 'with', 'to', 'he', 'she', 'or', 'a', 'that', 'in', 'is', 'on', 'for', 'our', 'by', 'its', 'my', 'your', 'you', 'me', 'i', 'it', 'whom', 'who', 'only', 'are', 'am', 'into', 'were', 'was', 'and', 'so', 'has', 'have', 'been', 'had', 'be', 'ours', 'ourselves', 'same', 'while', 'which', 'where', 'how', 'having', 'doing', 'myself', 'but', 'off', 'further', 'herself', 'himself', 'there', 'these', 'they', 'we', "you're", "you've", "you'll", "you'd", 'yours', 'yourself', 'yourselves', 'him', 'his', "she's", 'her', 'hers', "it's", 'itself', 'them', 'their', 'theirs', 'themselves', 'what', "that'll", 'those', 'being', 'if', 'because', 'as', 'until', 'about', 'against', 'between', 'through', 'during', 'before', 'after', 'above', 'below', 'from', 'up', 'down', 'out', 'over', 'under', 'again', 'then', 'once', 'here', 'when', 'why', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'own', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]

In [3]:
# convert categorical label to digits for processing
def labelCovert(label):
    if label == "neutral": return 0
    if label  == 'positive': return 1       #replace each label with numeric value
    elif label  == 'negative': return 2


In [4]:
def remove_punctuation(sentence):
    return sentence.translate(str.maketrans('', '', string.punctuation))  #remove puncutaiton from each tweet

In [5]:
# remove emoji for data cleaning
def removeEmojis(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)


In [6]:
def removeDigitsFromWords(word):
    return''.join([i for i in word if not i.isdigit()])

In [7]:
#return array of bag of words 
def bagOfWords(text, vocab):
    #sentence_words = extract_words(sentence)
    # frequency word count
    bag = np.zeros(len(vocab))
    for word in text:
        for i,w in enumerate(vocab):    #prepare bag of words
            if w == word: 
                bag[i] += 1
                
    return np.array(bag)

In [8]:
#load dataset
labels = []
texts = []
with open(dataset_dir,'r',  encoding='utf8') as file:     # read and seperate text and their labels
    reader = csv.reader(file)
    for row in reader:
        labels.append(row[0])
        texts.append(row[1])

labels = labels[1:]
texts = texts[1:]

labels = [labelCovert(label) for label in labels ]
labels, texts = zip(*sorted(zip(labels, texts)))

In [9]:
#clean data
def cleanData(texts):
    
    for i,_ in enumerate(texts):
        texts[i] = ' '.join(word for word in texts[i].split(' ') if not word.startswith('@')) #remove airline name
        
    texts = [str(np.char.lower(text)) for text in texts]  #lower case
    texts = [remove_punctuation(text) for text in texts]  # remove puncuation
    
    for i,_ in enumerate(texts):
        texts[i] = ' '.join(word for word in texts[i].split() if word not in stopWords) #remove stop words
        texts[i] = re.sub(r"http\S+", "", texts[i])  #remove URL
    
    texts = [removeEmojis(text) for text in texts]  # remove emojis
    texts = [ re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", text) for text in texts]  # remove digits
        
    return texts
     

In [10]:
#splits data into train and test in stratified way
def splitData(labels, texts , splitPercentage):   
    
    trainLabels =[]
    testLabels = []
    
    trainTexts = []
    testTexts = []
    
    neutralCount = labels.count(0)
    positiveCount = labels.count(1)
    negativeCount = labels.count(2)
    
    testNeutralCount = int(neutralCount * splitPercentage)
    testPositiveCount = int(positiveCount * splitPercentage)
    testNegativeCount = int(negativeCount * splitPercentage)

    
    testLabels = testLabels + list(labels[: (testNeutralCount)])
    testTexts = testTexts + list(texts[: (testNeutralCount)])
    
    trainLabels = trainLabels + list(labels[testNeutralCount : (neutralCount )])
    trainTexts = trainTexts +  list(texts[testNeutralCount : (neutralCount)])
    
    testPositiveCount =  neutralCount + testPositiveCount         

    
    testLabels = testLabels + list(labels[neutralCount : testPositiveCount])
    testTexts = testTexts + list(texts[ neutralCount : testPositiveCount])
    
    trainLabels = trainLabels + list(labels[testPositiveCount : (neutralCount+positiveCount)])
    trainTexts = trainTexts + list(texts[testPositiveCount : (neutralCount+positiveCount)])
    
    testNegativeCount =  neutralCount + positiveCount + testNegativeCount
    
    testLabels = testLabels + list(labels[(neutralCount+positiveCount) : testNegativeCount])
    testTexts = testTexts + list(texts[ (neutralCount+positiveCount ) : testNegativeCount])
    
    trainLabels = trainLabels + list(labels[testNegativeCount:])
    trainTexts = trainTexts + list(texts[testNegativeCount: ])
    
    return testLabels, testTexts ,trainLabels , trainTexts

In [11]:
testLabels, testTexts ,trainLabels , trainTexts =  splitData(labels, texts , 0.20)

In [12]:
#for bag of words
def preProcessTrainTextData(textTrainData,labels):
    
    
    trainTexts = cleanData(textTrainData)  # clean train tweets   
    tokenizeTrainText = [text.split() for text in trainTexts] # tokenize train tweets
 
    neutralText =  []
    positiveText = []
    negativeText = []
    
    for i,value in enumerate(labels):
        if value == 0:
            neutralText.append(tokenizeTrainText[i])
        elif value == 1:
            positiveText.append(tokenizeTrainText[i])
        elif value == 2:
            negativeText.append(tokenizeTrainText[i])

    neutralVocab = []
    posVocab = []
    negVocab = []
    
    for text in neutralText:
        neutralVocab = neutralVocab + text   # preparing list of all the  neutral words in train data
    
    for text in positiveText:
        posVocab = posVocab + text   # preparing list of all the positive words in train data
    
    for text in negativeText:
        negVocab = negVocab + text   # preparing list of all the negative words in train data
    
    neutralVocab = list(set(neutralVocab))
    posVocab = list(set(posVocab))
    negVocab = list(set(negVocab))
    
    
    totalVocab = list(set(neutralVocab + posVocab + negVocab))

    trainNeuDataList = [bagOfWords(text,totalVocab) for text in  neutralText]   #bag of words for newtral train data 
    trainNeuDataMatrix = np.vstack(trainNeuDataList)    # data Matrix for neutral train data
    
    trainPosDataList = [bagOfWords(text,totalVocab) for text in  positiveText]   #bag of words for positive train data 
    trainPosDataMatrix = np.vstack(trainPosDataList)    # data Matrix for Pos train data
    
    trainNegDataList = [bagOfWords(text,totalVocab) for text in  negativeText]   #bag of words for positive train data 
    trainNegDataMatrix = np.vstack(trainNegDataList)    # data Matrix for Pos train data

    return  trainNeuDataMatrix, trainPosDataMatrix, trainNegDataMatrix, neutralVocab,  posVocab , negVocab , totalVocab

In [13]:
trainNeuDataMatrix, trainPosDataMatrix, trainNegDataMatrix, neutralVocab,  posVocab , negVocab , totalVocab = preProcessTrainTextData(trainTexts, trainLabels)

$N_c$ be the number of documents in our training data with class c and $N_{doc}$ be the total number of documents
$$Prior = \frac{N_c}{N_{doc}}$$

$$Liklihood =\hat{P}(w_{i}|c)= \frac{count(w_{i}|c)+1}{\sum_{w\in V}count(w_{i}|c)+|V|} $$

In [14]:
def trainNaiveBayes(trainNeuDataMatrix, trainPosDataMatrix, trainNegDataMatrix,neutralVocab, posVocab , negVocab, totalVocab, labels, classes = [0,1,2] ):
    
    totalClasses = len(labels)
    prior = {}
    likelihood = {}
    
    for cls in classes:
        
        classCount = labels.count(cls)
        prior[cls]= classCount/totalClasses       # prior for each class
        
        if cls == 0:
            totalSum =  trainNeuDataMatrix.sum()
            totalSum = totalSum + len(totalVocab)
            
            for i, word in enumerate(totalVocab):
                likelihood[word,cls] = (np.sum(trainNeuDataMatrix[:,i]) + 1)/totalSum     # likelihood for neutral class
                
        elif cls == 1:
            totalSum =  trainPosDataMatrix.sum()
            totalSum = totalSum + len(totalVocab)
            
            for i, word in enumerate(totalVocab):
                likelihood[word,cls] = (np.sum(trainPosDataMatrix[:,i]) + 1)/totalSum     # likelihood for positive class
                
        elif cls == 2:
            totalSum =  trainNegDataMatrix.sum()
            totalSum = totalSum + len(totalVocab)
                    
            for i, word in enumerate(totalVocab):
                likelihood[word,cls] = (np.sum(trainNegDataMatrix[:,i]) + 1)/totalSum    # likelihood for negative class
         
    return prior, likelihood  
            

In [15]:
prior, likelihood =  trainNaiveBayes(trainNeuDataMatrix, trainPosDataMatrix, trainNegDataMatrix,neutralVocab, posVocab , negVocab, totalVocab, labels )

In [16]:
#Peprocess test data for prediction
def PreProcessTestData(testTexts):
    
    testTexts = cleanData(testTexts)  # clean test in tweets
    tokenizeTestText = [text.split() for text in testTexts] # tokenize test tweets
    
    return tokenizeTestText

In [17]:
#Call test data preprocessing
textData = PreProcessTestData(testTexts)

for each class c
$$sum[c] = logprior[c]$$

for each word of a text
$$if word\in V then \;\; sum[c] = sum[c]+ loglikelihood[word,c]$$

$$\hat{y} = argmax_c sum[c]$$

In [18]:
#Predict label of text data by using Prior and liklihood calculated from train data
def predictTestData(testData , totalVocab, prior, likelihood, classes = [0,1,2]):
    
    probabilities = []
    probability = np.zeros((1, len(classes)))
    predictedLabels =  np.zeros((len(testData),1))

    for i, tweet in enumerate(testData):
        summation = {}
        probability.fill(0)
        
        for cls in classes:
            summation[cls] = 1
            for word in tweet:
                if word in totalVocab:
                    summation[cls] = summation[cls] * likelihood[word,cls]   # summing liklihood of a tweet
            probability[:,cls] =  prior[cls] * summation[cls]             # calculating probability of a tweet for each class
        
        predictedLabels[i] = np.argmax(probability)     # gettings highest probablity for each tweet


    return  list(predictedLabels)

In [19]:
#Predict label for test data
pred_Y = predictTestData(textData, totalVocab, prior, likelihood)

In [20]:
# returns precision
def getPrecision(TP, FP):  
    return (TP)/(TP+FP)

In [21]:
# returns recall
def getRecall(TP,FN):   
    return (TP)/(TP+FN)

In [22]:
# returns accuracy
def getAccuracy(TP,  total):  
    return (TP)/(total )

In [23]:
# returns F1 Score
def getF1Score(precision,recall ):
    return (2*precision*recall)/(precision+recall)

In [24]:
#function for evalution
def perf_measure(y_actual, y_pred):
    TP_Neutral = 0
    FP_Neutral = 0
    TN_Neutral = 0
    FN_Neutral = 0
    
    TP_Positive = 0
    FP_Positive = 0
    TN_Positive = 0
    FN_Positive = 0
    
    TP_Negative = 0
    FP_Negative = 0
    TN_Negative = 0
    FN_Negative = 0
    

    for i in range(len(y_pred)): 
        if y_actual[i]==y_pred[i]==0:
            TP_Neutral  += 1
        if y_actual[i]==y_pred[i]==1:
            TP_Positive  += 1
        if y_actual[i]==y_pred[i]==2:
            TP_Negative += 1
        
        if  y_actual[i]==0 and y_pred[i]!=0:
            FN_Neutral += 1
        if y_actual[i] ==1 and y_pred[i]!=1:
            FN_Positive += 1
        if y_actual[i]==2 and y_pred[i]!=2:
            FN_Negative += 1
        
        if  y_pred[i]==0 and y_actual[i]!=y_pred[i]:
            FP_Neutral += 1
        if  y_pred[i]==1 and y_actual[i]!=y_pred[i]:
            FP_Positive += 1
        if  y_pred[i]==2 and y_actual[i]!=y_pred[i]:
            FP_Negative += 1
    
    
    precisionNeutral = getPrecision(TP_Neutral,FP_Neutral)
    precisionPositive = getPrecision(TP_Positive,FP_Positive)
    precisionNegative = getPrecision(TP_Negative,FP_Negative)
        
    recallNeutral = getRecall(TP_Neutral,FN_Neutral)
    recallPositive = getRecall(TP_Positive,FN_Positive)
    recallNegative =   getRecall(TP_Negative,FN_Negative)
        
    #accuray =  getAccuracy((TP_Neutral+TP_Positive+TP_Negative), len(y_pred))
    
    f1Neutral = getF1Score(precisionNeutral,recallNeutral)
    f1Positive = getF1Score(precisionPositive,recallPositive)
    f1Negative = getF1Score(precisionNegative,recallNegative)
    
    macroAverageF1Score = (f1Neutral+f1Positive+f1Negative)/3
    
        
    print(f"Precision of Neutral:{precisionNeutral}")
    print(f"Precision of Positive:{precisionPositive}")
    print(f"Precision of Negative:{precisionNegative}")
        
    print(f"Recall of Neutral:{recallNeutral}")
    print(f"Recall of Positive:{recallPositive}")
    print(f"Recall of Negative:{recallNegative}")
        
    #print(f"Overall Accuracy:{accuray}")
    print(f"Macro Average F1 Score:{macroAverageF1Score}")

In [25]:
# prints evaluation report
perf_measure(testLabels,pred_Y)  

Precision of Neutral:0.7310924369747899
Precision of Positive:0.8306709265175719
Precision of Negative:0.7494736842105263
Recall of Neutral:0.28109854604200324
Recall of Positive:0.5508474576271186
Recall of Negative:0.9700272479564033
Macro Average F1 Score:0.6380312536081728
